<a href="https://colab.research.google.com/github/rlepsch/argus/blob/main/argus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span style="color:blue">REINALDO LEPSCH NETO - TEST FOR ARGUS - 31/12/2021</span>


https://movile.blog/introducao-a-spark-usando-o-google-colab/

https://ichi.pro/pt/observacoes-sobre-como-salvar-dados-com-spark-3-0-148135666658161


# 1 Downloading files



In [1]:
#installing module wget
%%capture
!pip install -q wget
#creating folder to save output
!mkdir data

In [9]:
#downloading data from github (originally sent as email attachments)
import wget

url = "https://raw.githubusercontent.com/rlepsch/argus/main/repo1.csv"
wget.download(url, "data/repo1.csv")

url = "https://raw.githubusercontent.com/rlepsch/argus/main/repo2.csv"
wget.download(url, "data/repo2.csv")

url = "https://raw.githubusercontent.com/rlepsch/argus/main/vhf_code.csv"
wget.download(url, "data/vhf_code.csv")

'data/vhf_code (1).csv'

# 2 Apache Spark Cluster

In [3]:
#installing Java Runtime Environment (JRE) version 8
%%capture
!apt-get remove openjdk*
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
#downloading Apache Spark version 3.0.0
%%capture
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz && rm spark-3.0.0-bin-hadoop2.7.tgz

In [5]:
import os
#configuring the environment variable JAVA_HOME
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#configurando a variável de ambiente SPARK_HOME
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

In [6]:
%%capture
#installing package findspark
!pip install -q findspark==1.4.2
#instalando o pacote pyspark
!pip install -q pyspark==3.0.0

In [7]:
#importando o módulo findspark
import findspark
#carregando a variávels SPARK_HOME na variável dinâmica PYTHONPATH
findspark.init()

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("pyspark-notebook").master("local[*]").getOrCreate()

# 3 Data preparation


In [13]:
#creating and showing dataframe for repo1

repo1 = spark.read.csv(path="data/repo1.csv", header=True, sep=",")
repo1.show(5)


+-----------------+-----------+--------------------+------+---------------------+-------------------+--------------+------------+
|vhf_repository_id|vhf_code_id|publication_datetime| value|applies_from_datetime|applies_to_datetime|forward_period|forward_year|
+-----------------+-----------+--------------------+------+---------------------+-------------------+--------------+------------+
|        184950377|      45747|    15/12/2021 13:23|     2|     15/12/2021 13:22|   15/12/2021 13:23|             2|        2022|
|        184950374|      45743|    15/12/2021 13:23|2.0775|     15/12/2021 13:22|   15/12/2021 13:23|             2|        2022|
|        184950371|      45745|    15/12/2021 13:23|2.0775|     15/12/2021 13:22|   15/12/2021 13:23|             2|        2022|
|        184950368|      45744|    15/12/2021 13:23|2.0776|     15/12/2021 13:22|   15/12/2021 13:23|             2|        2022|
|        184950365|      45746|    15/12/2021 13:23|2.0776|     15/12/2021 13:22|   15/12/

In [14]:
#creating and showing dataframe for repo2

repo2 = spark.read.csv(path="data/repo2.csv", header=True, sep=",")
repo2.show(5)


+-----------------+-----------+--------------------+-----+---------------------+-------------------+--------------+------------+
|vhf_repository_id|vhf_code_id|publication_datetime|value|applies_from_datetime|applies_to_datetime|forward_period|forward_year|
+-----------------+-----------+--------------------+-----+---------------------+-------------------+--------------+------------+
|        184931068|      45691| 15/12/2021 12:38:00|72.65|  15/12/2021 12:37:00|15/12/2021 12:38:00|             4|        2022|
|        184931061|      45687| 15/12/2021 12:38:00|12.00|  15/12/2021 12:37:00|15/12/2021 12:38:00|             3|        2022|
|        184931059|      45683| 15/12/2021 12:38:00|72.93|  15/12/2021 12:37:00|15/12/2021 12:38:00|             3|        2022|
|        184931057|      45685| 15/12/2021 12:38:00|72.91|  15/12/2021 12:37:00|15/12/2021 12:38:00|             3|        2022|
|        184931055|      45684| 15/12/2021 12:38:00|72.93|  15/12/2021 12:37:00|15/12/2021 12:38:

In [15]:
#creating and showing dataframe for vhf_code

vhf_code = spark.read.csv(path="data/vhf_code.csv", header=True, sep=",")
vhf_code.show(5)


+--------------------+--------------------+--------------------+------------------+-----------+
|         description|external_description|   description_short|external_query_str|vhf_code_id|
+--------------------+--------------------+--------------------+------------------+-----------+
|Brent month 1 1-m...|     Crude Oil Brent|Brent M1 1-min close|             CLOSE|      45678|
|Brent month 1 1-m...|     Crude Oil Brent| Brent M1 1-min high|              HIGH|      45679|
|Brent month 1 1-m...|     Crude Oil Brent|  Brent M1 1-min low|               LOW|      45680|
|Brent month 1 1-m...|     Crude Oil Brent| Brent M1 1-min open|              OPEN|      45681|
|Brent month 1 1-m...|     Crude Oil Brent|Brent M1 1-min vo...|            VOLUME|      45682|
+--------------------+--------------------+--------------------+------------------+-----------+
only showing top 5 rows



In [16]:
#creating the temp views
repo1.createOrReplaceTempView("repo1")
repo2.createOrReplaceTempView("repo2")
vhf_code.createOrReplaceTempView("vhf_code")

In [ ]:
vhf_code

DataFrame[description: string, external_description: string, description_short: string, external_query_str: string, vhf_code_id: string]

# 4 Querying the data

In [17]:
query = """
SELECT vhf_repository_id
FROM repo1 
where value > 2
"""

spark.sql(query).show()

+-----------------+
|vhf_repository_id|
+-----------------+
|        184950361|
|        184950325|
|        184950320|
|        184950315|
|        184950310|
|        184950305|
|        184950299|
|        184950293|
|        184950287|
|        184950281|
|        184950275|
|        184950268|
|        184950262|
|        184950256|
|        184950250|
|        184950244|
|        184950376|
|        184950360|
|        184950324|
|        184950319|
+-----------------+
only showing top 20 rows



#5 Saving the data

In [18]:

( 
  repo1.write
  .mode('append') # or append
  #.partitionBy(col_name) # this is optional
  .bucketBy(5, 'forward_period') # n is number of buckets
  .sortBy('forward_period')
  #.format('parquet') # this is optional, parquet is default
  .option('path','file:///Users/reinaldolepsch/Downloads')
  .saveAsTable('tb_output')
)

In [19]:
df = spark.table('tb_output')

In [20]:
query = """
SELECT *
FROM tb_output 
"""

spark.sql(query).show()

+-----------------+-----------+--------------------+-----+---------------------+-------------------+--------------+------------+
|vhf_repository_id|vhf_code_id|publication_datetime|value|applies_from_datetime|applies_to_datetime|forward_period|forward_year|
+-----------------+-----------+--------------------+-----+---------------------+-------------------+--------------+------------+
|        184950144|      45692|    15/12/2021 13:22|   38|     15/12/2021 13:21|   15/12/2021 13:22|             4|        2022|
|        184950138|      45688|    15/12/2021 13:22|72.45|     15/12/2021 13:21|   15/12/2021 13:22|             4|        2022|
|        184950132|      45690|    15/12/2021 13:22|72.44|     15/12/2021 13:21|   15/12/2021 13:22|             4|        2022|
|        184950126|      45689|    15/12/2021 13:22|72.45|     15/12/2021 13:21|   15/12/2021 13:22|             4|        2022|
|        184950120|      45691|    15/12/2021 13:22|72.44|     15/12/2021 13:21|   15/12/2021 13:

# CELLS FOR THE TEST

## 5.3 Operação Pivot

**Definição:** Reorienta a visão multidimensional dos dados, oferecendo diferentes perspectivas dos mesmos dados.

**Exemplo de consulta pivot:** Qual o valor gasto em salários por nível de cargo, considerando cada ano?

In [ ]:
# utilizando a cláusula WHERE com subselects
query = """
SELECT anoPag AS `Ano`, ROUND(salario,2) AS `Total Gasto com Salários`, ROUND(receita,2) AS `Total de Receitas Recebidas`
FROM ( SELECT dataAno, SUM(salario) 
       FROM pagamento JOIN data on data.dataPK = pagamento.dataPK
       GROUP BY dataAno
      ) AS pag(anoPag, salario), 
     ( SELECT dataAno, SUM(receita)
       FROM negociacao JOIN data ON data.dataPK = negociacao.dataPK
       GROUP BY dataAno
      ) AS neg(anoNeg, receita) 
WHERE anoPag = anoNeg 
ORDER BY anoPag
"""

spark.sql(query).show()

+----+------------------------+---------------------------+
| Ano|Total Gasto com Salários|Total de Receitas Recebidas|
+----+------------------------+---------------------------+
|2016|              4442617.08|                 4614246.95|
|2017|               9775279.8|                 7200423.35|
|2018|            1.49355264E7|              1.159353965E7|
|2019|           1.856766636E7|               3.53533183E7|
|2020|           1.856766636E7|              3.022217595E7|
+----+------------------------+---------------------------+

